In [1]:
import pandas as pd
import os
import tensorflow as tf
import numpy as np
from tensorflow import keras

# For my Mac
os.chdir('//Users//patricklee//Desktop//파이썬//DACON//2020DACON_CUP')

# For my Desktop
# os.chdir('C://Users//BIS_COM//data//dacon//2020')

In [2]:
df = pd.read_csv(r'./new_train_df.csv', parse_dates=['c_time'])

In [3]:
df

,c_time,id,ds_level,country_code,Total visit,browser,platform,Total User,사용자,세션,신규방문자,페이지뷰
0,2018-09-09 01:00:00,1.0,1.0,1.0,-99.0,-99.0,-99.0,-99.0,20.0,19.0,9.0,259.0
1,2018-09-09 03:00:00,1.0,1.0,1.0,-99.0,-99.0,-99.0,-99.0,10.0,10.0,2.0,102.0
2,2018-09-09 13:00:00,3.0,1.0,1.0,-99.0,-99.0,-99.0,-99.0,20.0,16.0,7.0,131.0
3,2018-09-09 18:00:00,1.0,1.0,1.0,-99.0,-99.0,-99.0,-99.0,13.0,12.0,2.0,47.0
4,2018-09-09 21:00:00,2.0,1.0,1.0,-99.0,-99.0,-99.0,-99.0,17.0,17.0,7.0,107.0
...,...,...,...,...,...,...,...,...,...,...,...,...
13454,2019-12-30 07:00:00,-99.0,-99.0,-99.0,1.0,1.0,1.0,1.0,12.0,14.0,3.0,72.0
13455,2019-12-30 08:00:00,-99.0,-99.0,-99.0,7.0,3.0,3.0,6.0,29.0,29.0,5.0,274.0
13456,2019-12-30 12:00:00,-99.0,-99.0,-99.0,16.0,5.0,4.0,16.0,58.0,59.0,18.0,624.0
13457,2019-12-30 15:00:00,-99.0,-99.0,-99.0,36.0,7.0,5.0,34.0,100.0,100.0,28.0,1196.0


In [4]:
df = df.set_index("c_time")

In [5]:
df

,id,ds_level,country_code,Total visit,browser,platform,Total User,사용자,세션,신규방문자,페이지뷰
c_time,,,,,,,,,,,
2018-09-09 01:00:00,1.0,1.0,1.0,-99.0,-99.0,-99.0,-99.0,20.0,19.0,9.0,259.0
2018-09-09 03:00:00,1.0,1.0,1.0,-99.0,-99.0,-99.0,-99.0,10.0,10.0,2.0,102.0
2018-09-09 13:00:00,3.0,1.0,1.0,-99.0,-99.0,-99.0,-99.0,20.0,16.0,7.0,131.0
2018-09-09 18:00:00,1.0,1.0,1.0,-99.0,-99.0,-99.0,-99.0,13.0,12.0,2.0,47.0
2018-09-09 21:00:00,2.0,1.0,1.0,-99.0,-99.0,-99.0,-99.0,17.0,17.0,7.0,107.0
...,...,...,...,...,...,...,...,...,...,...,...
2019-12-30 07:00:00,-99.0,-99.0,-99.0,1.0,1.0,1.0,1.0,12.0,14.0,3.0,72.0
2019-12-30 08:00:00,-99.0,-99.0,-99.0,7.0,3.0,3.0,6.0,29.0,29.0,5.0,274.0
2019-12-30 12:00:00,-99.0,-99.0,-99.0,16.0,5.0,4.0,16.0,58.0,59.0,18.0,624.0


# 예제 첫 번째

In [6]:
epochs = 200
batch_size = 128
verbose = 1
nb_classes = 4
n_hidden = 128
validation_split = 0.2

rate = 0.2
rate_size = int(len(df) * rate)

train = df[:-rate_size]
test = df[-rate_size:]

In [15]:
x_col = train.columns[:7]
y_col = train.columns[7:]
print(x_col, y_col)

Index(['id', 'ds_level', 'country_code', 'Total visit', 'browser', 'platform',
       'Total User'],
      dtype='object') Index(['사용자', '세션', '신규방문자', '페이지뷰'], dtype='object')


In [16]:
X_train = train[x_col]
Y_train = train[y_col]
X_test = test[x_col]
Y_test = test[y_col]

In [17]:
print(X_train.shape, X_test.shape)

(10768, 7) (2691, 7)


In [18]:
model = tf.keras.models.Sequential()
model.add(keras.layers.Dense(nb_classes, input_shape=(7,)
                            ))

In [19]:
model.compile(optimizer = 'SGD',
             loss = 'MSE',
             metrics=['accuracy'])

In [20]:
model.fit(X_train,Y_train,
         batch_size=batch_size, epochs=epochs,
         verbose=verbose, validation_split=validation_split)

Epoch 1/200
68/68 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.1401 - val_loss: nan - val_accuracy: 0.0065
Epoch 2/200
68/68 [==============================] - 0s 677us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 3/200
68/68 [==============================] - 0s 660us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 4/200
68/68 [==============================] - 0s 674us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 5/200
68/68 [==============================] - 0s 645us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 6/200
68/68 [==============================] - 0s 666us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 7/200
68/68 [==============================] - 0s 727us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 8/200
68/68 [========================

68/68 [==============================] - 0s 628us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 61/200
68/68 [==============================] - 0s 628us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 62/200
68/68 [==============================] - 0s 622us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 63/200
68/68 [==============================] - 0s 616us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 64/200
68/68 [==============================] - 0s 627us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 65/200
68/68 [==============================] - 0s 627us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 66/200
68/68 [==============================] - 0s 620us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 67/200
68/68 [===========================

68/68 [==============================] - 0s 620us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 120/200
68/68 [==============================] - 0s 623us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 121/200
68/68 [==============================] - 0s 615us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 122/200
68/68 [==============================] - 0s 617us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 123/200
68/68 [==============================] - 0s 620us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 124/200
68/68 [==============================] - 0s 638us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 125/200
68/68 [==============================] - 0s 655us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 126/200
68/68 [====================

68/68 [==============================] - 0s 625us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 179/200
68/68 [==============================] - 0s 625us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 180/200
68/68 [==============================] - 0s 630us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 181/200
68/68 [==============================] - 0s 628us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 182/200
68/68 [==============================] - 0s 619us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 183/200
68/68 [==============================] - 0s 633us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 184/200
68/68 [==============================] - 0s 626us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 185/200
68/68 [====================

In [21]:
test_loss, test_acc = model.evaluate(X_test,Y_test)
print('Test ACC : ', test_acc)

85/85 [==============================] - 0s 365us/step - loss: nan - accuracy: 0.0000e+00
Test ACC :  0.0


# 두 번째 예제

In [22]:
model = tf.keras.models.Sequential()
model.add(keras.layers.Dense(n_hidden, input_shape=(7,),
                            activation='relu'))
model.add(keras.layers.Dense(n_hidden,
                            activation='relu'))
model.add(keras.layers.Dense(nb_classes,
                            activation='relu'))

In [23]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 128)               1024      
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 516       
Total params: 18,052
Trainable params: 18,052
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.compile(optimizer = 'SGD',
             loss = 'MSE',
             metrics=['accuracy'])

In [25]:
model.fit(X_train,Y_train,
         batch_size=batch_size, epochs=epochs,
         verbose=verbose, validation_split=validation_split)

Epoch 1/200
68/68 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0089 - val_loss: nan - val_accuracy: 0.0065
Epoch 2/200
68/68 [==============================] - 0s 868us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 3/200
68/68 [==============================] - 0s 862us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 4/200
68/68 [==============================] - 0s 867us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 5/200
68/68 [==============================] - 0s 892us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 6/200
68/68 [==============================] - 0s 890us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 7/200
68/68 [==============================] - 0s 873us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 8/200
68/68 [========================

68/68 [==============================] - 0s 850us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 61/200
68/68 [==============================] - 0s 835us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 62/200
68/68 [==============================] - 0s 828us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 63/200
68/68 [==============================] - 0s 836us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 64/200
68/68 [==============================] - 0s 829us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 65/200
68/68 [==============================] - 0s 850us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 66/200
68/68 [==============================] - 0s 865us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 67/200
68/68 [===========================

68/68 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 120/200
68/68 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 121/200
68/68 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 122/200
68/68 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 123/200
68/68 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 124/200
68/68 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 125/200
68/68 [==============================] - 0s 984us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 126/200
68/68 [==============================] 

68/68 [==============================] - 0s 870us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 179/200
68/68 [==============================] - 0s 886us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 180/200
68/68 [==============================] - 0s 841us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 181/200
68/68 [==============================] - 0s 842us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 182/200
68/68 [==============================] - 0s 867us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 183/200
68/68 [==============================] - 0s 859us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 184/200
68/68 [==============================] - 0s 844us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 185/200
68/68 [====================

In [26]:
test_loss, test_acc = model.evaluate(X_test,Y_test)
print('Test ACC : ', test_acc)

85/85 [==============================] - 0s 474us/step - loss: nan - accuracy: 0.0000e+00
Test ACC :  0.0


# 세 번째 예제

In [27]:
model = tf.keras.models.Sequential()
model.add(keras.layers.Dense(n_hidden, input_shape=(7,),
                            activation='relu'))
model.add(keras.layers.Dense(n_hidden,
                            activation='relu'))
model.add(keras.layers.Dense(nb_classes))

In [28]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 128)               1024      
_________________________________________________________________
dense_6 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_7 (Dense)              (None, 4)                 516       
Total params: 18,052
Trainable params: 18,052
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.compile(optimizer = 'adam',
             loss = 'MSE',
             metrics=['accuracy'])

In [30]:
model.fit(X_train,Y_train,
         batch_size=batch_size, epochs=epochs,
         verbose=verbose, validation_split=validation_split)

Epoch 1/200
68/68 [==============================] - 0s 2ms/step - loss: 1234363.2500 - accuracy: 0.9922 - val_loss: 4454.0054 - val_accuracy: 0.9935
Epoch 2/200
68/68 [==============================] - 0s 1ms/step - loss: 557176.7500 - accuracy: 0.9928 - val_loss: 4128.3950 - val_accuracy: 0.9935
Epoch 3/200
68/68 [==============================] - 0s 965us/step - loss: 550041.6875 - accuracy: 0.9928 - val_loss: 4070.9927 - val_accuracy: 0.9935
Epoch 4/200
68/68 [==============================] - 0s 1ms/step - loss: 546864.4375 - accuracy: 0.9928 - val_loss: 3450.4482 - val_accuracy: 0.9935
Epoch 5/200
68/68 [==============================] - 0s 1ms/step - loss: 543251.4375 - accuracy: 0.9928 - val_loss: 3468.5078 - val_accuracy: 0.9935
Epoch 6/200
68/68 [==============================] - 0s 1ms/step - loss: 539143.3125 - accuracy: 0.9928 - val_loss: 3459.3523 - val_accuracy: 0.9935
Epoch 7/200
68/68 [==============================] - 0s 1ms/step - loss: 533700.2500 - accuracy: 0.9928

68/68 [==============================] - 0s 922us/step - loss: 358730.8438 - accuracy: 0.9928 - val_loss: 2896.4688 - val_accuracy: 0.9935
Epoch 56/200
68/68 [==============================] - 0s 904us/step - loss: 360754.0625 - accuracy: 0.9894 - val_loss: 3120.1768 - val_accuracy: 0.9935
Epoch 57/200
68/68 [==============================] - 0s 916us/step - loss: 357420.2188 - accuracy: 0.9593 - val_loss: 3002.8157 - val_accuracy: 0.9935
Epoch 58/200
68/68 [==============================] - 0s 910us/step - loss: 359626.9375 - accuracy: 0.9850 - val_loss: 3694.6453 - val_accuracy: 0.7094
Epoch 59/200
68/68 [==============================] - 0s 921us/step - loss: 357512.2188 - accuracy: 0.9885 - val_loss: 2928.2288 - val_accuracy: 0.9935
Epoch 60/200
68/68 [==============================] - 0s 936us/step - loss: 358554.7188 - accuracy: 0.9928 - val_loss: 2921.4309 - val_accuracy: 0.9935
Epoch 61/200
68/68 [==============================] - 0s 921us/step - loss: 358055.4688 - accuracy: 0

68/68 [==============================] - 0s 1ms/step - loss: 357877.5938 - accuracy: 0.9928 - val_loss: 3018.9624 - val_accuracy: 0.9935
Epoch 110/200
68/68 [==============================] - 0s 991us/step - loss: 360250.9375 - accuracy: 0.9928 - val_loss: 3128.4590 - val_accuracy: 0.9935
Epoch 111/200
68/68 [==============================] - 0s 901us/step - loss: 358240.6875 - accuracy: 0.9927 - val_loss: 3310.4404 - val_accuracy: 0.9935
Epoch 112/200
68/68 [==============================] - 0s 894us/step - loss: 356359.6562 - accuracy: 0.9928 - val_loss: 3356.4741 - val_accuracy: 0.9935
Epoch 113/200
68/68 [==============================] - 0s 904us/step - loss: 355954.7188 - accuracy: 0.9928 - val_loss: 3064.1396 - val_accuracy: 0.9935
Epoch 114/200
68/68 [==============================] - 0s 897us/step - loss: 356877.8750 - accuracy: 0.9900 - val_loss: 3153.5774 - val_accuracy: 0.9935
Epoch 115/200
68/68 [==============================] - 0s 890us/step - loss: 357868.6250 - accurac

Epoch 163/200
68/68 [==============================] - 0s 907us/step - loss: 355199.2188 - accuracy: 0.9928 - val_loss: 4248.6299 - val_accuracy: 0.7159
Epoch 164/200
68/68 [==============================] - 0s 922us/step - loss: 356049.3438 - accuracy: 0.9836 - val_loss: 3111.1008 - val_accuracy: 0.9935
Epoch 165/200
68/68 [==============================] - 0s 916us/step - loss: 359916.5312 - accuracy: 0.9882 - val_loss: 3290.1411 - val_accuracy: 0.9935
Epoch 166/200
68/68 [==============================] - 0s 993us/step - loss: 356763.2188 - accuracy: 0.9928 - val_loss: 3897.7476 - val_accuracy: 0.9935
Epoch 167/200
68/68 [==============================] - 0s 1ms/step - loss: 356266.9688 - accuracy: 0.9928 - val_loss: 3155.5400 - val_accuracy: 0.9935
Epoch 168/200
68/68 [==============================] - 0s 1ms/step - loss: 357302.5312 - accuracy: 0.9928 - val_loss: 3080.7725 - val_accuracy: 0.9935
Epoch 169/200
68/68 [==============================] - 0s 978us/step - loss: 358320.12

In [31]:
test_loss, test_acc = model.evaluate(X_test,Y_test)
print('Test ACC : ', test_acc)

85/85 [==============================] - 0s 405us/step - loss: 12574.5400 - accuracy: 1.0000
Test ACC :  1.0


In [32]:
model.compile(optimizer = 'RMSProp',
             loss = 'MSE',
             metrics=['accuracy'])

In [33]:
model.fit(X_train,Y_train,
         batch_size=batch_size, epochs=epochs,
         verbose=verbose, validation_split=validation_split)

Epoch 1/200
68/68 [==============================] - 0s 2ms/step - loss: 357746.1562 - accuracy: 0.9921 - val_loss: 4086.6621 - val_accuracy: 0.9935
Epoch 2/200
68/68 [==============================] - 0s 939us/step - loss: 358670.9375 - accuracy: 0.9928 - val_loss: 3657.4036 - val_accuracy: 0.9935
Epoch 3/200
68/68 [==============================] - 0s 935us/step - loss: 358037.1875 - accuracy: 0.9928 - val_loss: 3491.0017 - val_accuracy: 0.9935
Epoch 4/200
68/68 [==============================] - 0s 943us/step - loss: 357575.5000 - accuracy: 0.9928 - val_loss: 3537.5002 - val_accuracy: 0.9935
Epoch 5/200
68/68 [==============================] - 0s 943us/step - loss: 357709.0625 - accuracy: 0.9923 - val_loss: 3997.9229 - val_accuracy: 0.9935
Epoch 6/200
68/68 [==============================] - 0s 947us/step - loss: 356073.1250 - accuracy: 0.9916 - val_loss: 4064.8718 - val_accuracy: 0.9935
Epoch 7/200
68/68 [==============================] - 0s 922us/step - loss: 358564.6875 - accurac

68/68 [==============================] - 0s 943us/step - loss: 356892.6250 - accuracy: 0.9928 - val_loss: 4386.1045 - val_accuracy: 0.7159
Epoch 56/200
68/68 [==============================] - 0s 964us/step - loss: 356412.0938 - accuracy: 0.9923 - val_loss: 3298.1541 - val_accuracy: 0.9935
Epoch 57/200
68/68 [==============================] - 0s 978us/step - loss: 357140.0625 - accuracy: 0.9921 - val_loss: 3984.0825 - val_accuracy: 0.9935
Epoch 58/200
68/68 [==============================] - 0s 865us/step - loss: 356804.1875 - accuracy: 0.9923 - val_loss: 3894.4880 - val_accuracy: 0.9935
Epoch 59/200
68/68 [==============================] - 0s 960us/step - loss: 356633.0000 - accuracy: 0.9918 - val_loss: 3310.8982 - val_accuracy: 0.9935
Epoch 60/200
68/68 [==============================] - 0s 872us/step - loss: 356949.8438 - accuracy: 0.9919 - val_loss: 3379.5093 - val_accuracy: 0.9935
Epoch 61/200
68/68 [==============================] - 0s 862us/step - loss: 356288.3125 - accuracy: 0

Epoch 109/200
68/68 [==============================] - 0s 905us/step - loss: 355089.6875 - accuracy: 0.9920 - val_loss: 3062.1533 - val_accuracy: 0.9935
Epoch 110/200
68/68 [==============================] - 0s 917us/step - loss: 356349.2812 - accuracy: 0.9919 - val_loss: 3628.4443 - val_accuracy: 0.9935
Epoch 111/200
68/68 [==============================] - 0s 892us/step - loss: 354878.5312 - accuracy: 0.9928 - val_loss: 3236.9202 - val_accuracy: 0.9935
Epoch 112/200
68/68 [==============================] - 0s 912us/step - loss: 356058.1250 - accuracy: 0.9915 - val_loss: 4272.4316 - val_accuracy: 0.7159
Epoch 113/200
68/68 [==============================] - 0s 878us/step - loss: 356170.1875 - accuracy: 0.9912 - val_loss: 3163.1899 - val_accuracy: 0.9935
Epoch 114/200
68/68 [==============================] - 0s 878us/step - loss: 356491.0625 - accuracy: 0.9920 - val_loss: 4086.5684 - val_accuracy: 0.9935
Epoch 115/200
68/68 [==============================] - 0s 865us/step - loss: 35557

Epoch 163/200
68/68 [==============================] - 0s 866us/step - loss: 355735.7188 - accuracy: 0.9928 - val_loss: 3654.1958 - val_accuracy: 0.9935
Epoch 164/200
68/68 [==============================] - 0s 886us/step - loss: 355884.9062 - accuracy: 0.9897 - val_loss: 2928.6633 - val_accuracy: 0.9935
Epoch 165/200
68/68 [==============================] - 0s 872us/step - loss: 357527.9062 - accuracy: 0.9923 - val_loss: 2935.1106 - val_accuracy: 0.9935
Epoch 166/200
68/68 [==============================] - 0s 864us/step - loss: 355583.4688 - accuracy: 0.9922 - val_loss: 3490.6619 - val_accuracy: 0.9935
Epoch 167/200
68/68 [==============================] - 0s 870us/step - loss: 355771.1875 - accuracy: 0.9911 - val_loss: 5282.7046 - val_accuracy: 0.5320
Epoch 168/200
68/68 [==============================] - 0s 879us/step - loss: 355115.5938 - accuracy: 0.9894 - val_loss: 3496.1667 - val_accuracy: 0.9935
Epoch 169/200
68/68 [==============================] - 0s 877us/step - loss: 35694

In [34]:
test_loss, test_acc = model.evaluate(X_test,Y_test)
print('Test ACC : ', test_acc)

85/85 [==============================] - 0s 456us/step - loss: 12684.4121 - accuracy: 1.0000
Test ACC :  1.0
